In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PacTimeOrig.controllers import simulator as sim
from PacTimeOrig.controllers import JaxMod as jm
from PacTimeOrig.controllers import models as mods

from PacTimeOrig.controllers import utils as ut
from PacTimeOrig.data import scripts
from PacTimeOrig.controllers.Scripts import SimulationScript as simrun


cfgparams={'area':'dACC'}
cfgparams['session']=2
cfgparams['subj']='H'
cfgparams['rbfs']=[20,30]
cfgparams['models']=['p','pv','pf','pvi','pif','pvf','pvif']
cfgparams['gpscaler']=[1,3,8]
cfgparams['restarts']=3
cfgparams['slack']=False
cfgparams['trials']=5
cfgparams['ngains']=[1,2,2,3,3,3,4] 
cfgparams['opttype']=['second']# Adam and Lbfgs
cfgparams['confusiontest']=False# Adam and Lbfgs
cfgparams['alpha']=5# Adam and Lbfgs
cfgparams['scaling']=0.001

# simrun.simulate(cfgparams)

routine: 
1. Fit a model 5x, store loss, elbo, trajectories
2. Store the one with the best loss (store its elbo, trajectories and loss and cov matrix and parameters)
3. Store best chosen model with BMS, do BMA over trajectories.

In [ ]:
prior_std={'weights': 10, 'widths': 3, 'gains': 2}

# Use monkey data
Xdsgn, kinematics, sessvars, psth = scripts.monkey_run(cfgparams)

# trials to use
trialidx = np.sort(np.random.randint(1, len(Xdsgn), cfgparams['trials']))

# Get system parameters
A, B = ut.define_system_parameters()
num_rbfs=20
modname='pv'
ngains=2
trial=14


# Get data
tdat = ut.trial_grab_kine(Xdsgn, trial)

#generate gains
L1,L2=ut.generate_sim_gains(ngains)


outputs = sim.controller_sim_pv(tdat, 6, L1, L2, A=None, B=None, gpscaler=1,assignment='soft')


tdat['player_pos']=outputs['x'][:,:2]
tdat['player_vel']=outputs['x'][:,2:]

# Make time
tmp = ut.make_timeline(outputs)

#Prep inputs
inputs = ut.prepare_inputs(A, B, outputs['x'], outputs['uout'], tdat['pry1_pos'], tdat['pry2_pos'], tmp, num_rbfs,
                           outputs['x'][:, 2:], tdat['pry1_vel'], tdat['pry2_vel'], pry_1_accel=tdat['pry1_accel'],
                           pry_2_accel=tdat['pry2_accel'])

#choose loss
loss_function = jm.create_loss_function_inner_bayes(ut.generate_rbf_basis, num_rbfs,
                                              ut.generate_smoothing_penalty, lambda_reg=0.1,
                                              ctrltype=modname,use_gmf_prior=True,
    prior_std=prior_std)



#only used for trust
grad_loss = ut.compute_loss_gradient(loss_function)
hess_loss = ut.compute_hessian(loss_function)


params, best_params_flat, best_loss = jm.outer_optimization_lbfgs(inputs, loss_function, grad_loss,hess_loss,randomize_weights=True, ctrltype=modname, maxiter=3000, tolerance=1e-5, optimizer='trust',  slack_model=cfgparams['slack'],bayes=True)                                                                                  

prior_hessian=jm.compute_prior_hessian( prior_std=prior_std, lambda_reg=0.1, num_weights=num_rbfs, num_gains=2*ngains, smoothing_matrix=ut.generate_smoothing_penalty(num_rbfs))


cov_matrix = jm.compute_posterior_covariance(hess_loss, best_params_flat, inputs, prior_hessian)

controller_trajectories = jm.simulate_posterior_samples(best_params_flat, cov_matrix, inputs)

#Compute the elbo
jm.compute_elbo(prior_std, best_params_flat, cov_matrix, inputs, modname, num_samples=300)



weights = params[2]
width = params[3]
# transform paramteres to correct domain
L1_fit = np.array(params[0])
L2_fit = np.array(params[1])

wtsim = ut.generate_sim_switch(inputs, width, weights,slack_model=cfgparams['slack'])

shift = np.vstack((wtsim[0], wtsim[1]))

Try it with monkey data now:

In [1]:
prior_std={'weights': 10, 'widths': 3, 'gains': 2}

# Use monkey data
Xdsgn, kinematics, sessvars, psth = scripts.monkey_run(cfgparams)

# trials to use
trialidx = np.sort(np.random.randint(1, len(Xdsgn), cfgparams['trials']))

# Get system parameters
A, B = ut.define_system_parameters()
num_rbfs=20
lambda_reg=5
modname='pvi'
ngains=3
trial=15


# Get data

tdat=ut.get_data_for_fit(Xdsgn, trial)

tdat['x']=np.hstack((tdat['player_pos'],tdat['player_vel']))


# Make time
tmp = ut.make_timeline(tdat)



#Prep inputs
inputs = ut.prepare_inputs(A, B, tdat['x'], tdat['uout'], tdat['pry1_pos'], tdat['pry2_pos'], tmp, num_rbfs,
                           tdat['x'][:, 2:], tdat['pry1_vel'], tdat['pry2_vel'], pry_1_accel=tdat['pry1_accel'],
                           pry_2_accel=tdat['pry2_accel'])

#choose loss
loss_function = jm.create_loss_function_inner_bayes(ut.generate_rbf_basis, num_rbfs,
                                              ut.generate_smoothing_penalty, lambda_reg=lambda_reg,
                                              ctrltype=modname,use_gmf_prior=True,
    prior_std=prior_std)



#only used for trust
grad_loss = ut.compute_loss_gradient(loss_function)
hess_loss = ut.compute_hessian(loss_function)


params, best_params_flat, best_loss = jm.outer_optimization_lbfgs(inputs, loss_function, grad_loss,hess_loss,randomize_weights=True, ctrltype=modname, maxiter=3000, tolerance=1e-5, optimizer='trust',  slack_model=cfgparams['slack'],bayes=True)                                                                                  

prior_hessian=jm.compute_prior_hessian( prior_std=prior_std, lambda_reg=lambda_reg, num_weights=num_rbfs, num_gains=2*ngains, smoothing_matrix=ut.generate_smoothing_penalty(num_rbfs))


cov_matrix = jm.compute_posterior_covariance(hess_loss, best_params_flat, inputs, prior_hessian)

controller_trajectories = jm.simulate_posterior_samples(best_params_flat, cov_matrix, inputs)

#Compute the elbo
jm.compute_elbo(prior_std, best_params_flat, cov_matrix, inputs, modname, num_samples=100)



weights = params[2]
width = params[3]
# transform paramteres to correct domain
L1_fit = np.array(params[0])
L2_fit = np.array(params[1])

wtsim = ut.generate_sim_switch(inputs, width, weights,slack_model=cfgparams['slack'])

shift = np.vstack((wtsim[0], wtsim[1]))

NameError: name 'scripts' is not defined

In [ ]:
cfgparams['folder']='/Users/user/PycharmProjects/PacManMain/data/HumanEMU'
subjects=os.listdir(cfgparams['folder'])
cfgparams['subj']=subjects[3]
Xdsgn, kinematics, sessvars, psth=scripts.human_emu_run(cfgparams)

prior_std={'weights': 10, 'widths': 3, 'gains': 2}

# Use monkey data

# trials to use
trialidx = np.sort(np.random.randint(1, len(Xdsgn), cfgparams['trials']))

# Get system parameters
A, B = ut.define_system_parameters()
num_rbfs=21
lambda_reg=3
modname='pf'
ngains=2
trial=26



# Get data

tdat=ut.get_data_for_fit(Xdsgn, trial)

tdat['x']=np.hstack((tdat['player_pos'],tdat['player_vel']))


# Make time
tmp = ut.make_timeline(tdat)



#Prep inputs
inputs = ut.prepare_inputs(A, B, tdat['x'], tdat['uout'], tdat['pry1_pos'], tdat['pry2_pos'], tmp, num_rbfs,
                           tdat['x'][:, 2:], tdat['pry1_vel'], tdat['pry2_vel'], pry_1_accel=tdat['pry1_accel'],
                           pry_2_accel=tdat['pry2_accel'])

#choose loss
loss_function = jm.create_loss_function_inner_bayes(ut.generate_rbf_basis, num_rbfs,
                                              ut.generate_smoothing_penalty, lambda_reg=lambda_reg,
                                              ctrltype=modname,use_gmf_prior=True,
    prior_std=prior_std)



#only used for trust
grad_loss = ut.compute_loss_gradient(loss_function)
hess_loss = ut.compute_hessian(loss_function)


params, best_params_flat, best_loss = jm.outer_optimization_lbfgs(inputs, loss_function, grad_loss,hess_loss,randomize_weights=True, ctrltype=modname, maxiter=3000, tolerance=1e-5, optimizer='trust',  slack_model=cfgparams['slack'],bayes=True)                                                                                  

prior_hessian=jm.compute_prior_hessian( prior_std=prior_std, lambda_reg=lambda_reg, num_weights=num_rbfs, num_gains=2*ngains, smoothing_matrix=ut.generate_smoothing_penalty(num_rbfs))

cov_matrix = jm.compute_posterior_covariance(hess_loss, best_params_flat, inputs, prior_hessian)

controller_trajectories = jm.simulate_posterior_samples(best_params_flat, cov_matrix, inputs)

#Compute the elbo
jm.compute_elbo(prior_std, best_params_flat, cov_matrix, inputs, modname, num_samples=50)